In [29]:
import sys
import os

# Get the parent directory of the "notebook" folder
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add it to the Python path
sys.path.append(project_root)

In [31]:
from strategies.moving_average import MovingAverage
from strategies.momentum import MomentumStrategy
from utils.backtest import Backtest
from utils.data_handler import DataHandler
import yfinance as yf
import pandas as pd
import matplotlib as plt

In [33]:
import pandas as pd
from itertools import product

# Define tickers and parameter grid
tickers = tickers = [
    "AAPL", "ABT", "ADI", "ADM", "ADP", "ADSK", "AEE", "AEP", "AJG", "AKAM",
    "ALB", "AMAT", "AMD", "AME", "AMGN", "AMT", "AMZN", "AON", "AOS", "APA",
    "APD", "APH", "ARE", "AVB", "AVY", "AXP", "AZO", "BA", "BAC", "BAX",
    "BBY", "BDX", "BEN", "BIIB", "BK", "BLK", "BMY", "BSX", "BXP", "C",
    "CAG", "CAH", "CAT", "CB", "CCI", "CDNS", "CHD", "CHRW", "CINF", "CL",
    "CLX", "CMI", "CNP", "COO", "COP", "COST", "CPB", "CPRT", "CSCO", "CTAS",
    "CTSH", "D", "DE", "DGX", "DHI", "DIS", "DLTR", "DOV", "DRI", "DTE",
    "DVA", "EA", "EBAY", "ECL", "ED", "EFX", "EIX", "EL", "EMN", "EMR",
    "EOG", "EQR", "ES", "ESS", "F", "FAST", "FCX", "FDX", "FE", "FFIV",
    "FITB", "FRT", "GD", "GE", "GILD", "GIS", "GPC", "GWW", "HAL", "HAS"
]  # Extend list as needed

# Parameter grid
short_windows = [5, 10, 20, 30, 50]
long_windows = [50, 100, 150, 200, 300]
invert_flags = [True, False]

# Store results
all_results = {}
best_params = []


In [ ]:
# Step 1: Loop over all tickers and store all combinations
for ticker in tickers:
    print(f"\n--- Evaluating {ticker} ---")
    data_handler = DataHandler(ticker=ticker, start_date='2010-01-01', end_date='2020-01-01')
    data = data_handler.fetch_data()

    results = []

    for short_w, long_w, invert in product(short_windows, long_windows, invert_flags):
        if short_w >= long_w:
            continue  # Skip invalid combo

        strategy = MovingAverage(short_window=short_w, long_window=long_w, invert_signals=invert, mode='crossover')
        backtest = Backtest(data, strategy, plot_results=False)

        try:
            result = backtest.run()
            results.append({
                'Ticker': ticker,
                'Short Window': short_w,
                'Long Window': long_w,
                'Invert': invert,
                **result
            })
        except Exception as e:
            print(f"Error on {ticker} | short={short_w}, long={long_w}, invert={invert}: {e}")
            continue

    all_results[ticker] = results

# Step 2: Flatten all results and build one DataFrame
all_results_flat = [item for sublist in all_results.values() for item in sublist]
all_results_df = pd.DataFrame(all_results_flat)

[*********************100%***********************]  1 of 1 completed


--- Evaluating AAPL ---



[*********************100%***********************]  1 of 1 completed


--- Evaluating ABT ---



[*********************100%***********************]  1 of 1 completed

...


--- Evaluating AJG ---



[*********************100%***********************]  1 of 1 completed


--- Evaluating AKAM ---



[*********************100%***********************]  1 of 1 completed


--- Evaluating ALB ---


In [ ]:
# Step 3: Aggregate mean Sharpe ratio per parameter combination
aggregated = (
    all_results_df
    .groupby(['Short Window', 'Long Window', 'Invert'])
    .agg({
        'Sharpe Ratio': 'mean',
        'Return': 'mean',
        'Final Value': 'mean',
        'Max Drawdown': 'mean'
    })
    .reset_index()
    .sort_values(by='Sharpe Ratio', ascending=False)
)

# Display best parameter combinations
print("\n=== Aggregated Results by Parameter Combination (Sorted by Sharpe Ratio) ===")
print(aggregated.round(4))

# Optional: Save results
all_results_df.to_csv("all_backtest_results.csv", index=False)
aggregated.to_csv("aggregated_parameter_results.csv", index=False)